<h1 style="font-size: 22px; line-height: 100%; text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142); border-radius: 10px;">
External Sub-network Search with BootstrapNAS and DyNAS-T
</h1>

This notebook demonstrates how to use an external search solution to find high-performing subnetworks in NNCF's [BootstrapNAS](https://github.com/openvinotoolkit/nncf/tree/develop/examples/experimental/torch/classification). We will use an existing super-network generated from ResNet-50.  

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Imports and Settings
</h3>

In [2]:
import sys
import os
sys.path.append('..')
sys.path.append('/localdisk/maciej/code/dynast_bootstrapnas_integration/nncf/')
print(sys.path)

from imports_bnas import * # Import NNCF, PyTorch and other required packages.
import os
from examples.common.sample_config import SampleConfig  # Have to append path to NNCF to this import to work.
from nncf.experimental.torch.nas.bootstrapNAS.search.supernet import TrainedSuperNet
from nncf.experimental.torch.nas.bootstrapNAS.elasticity.multi_elasticity_handler import SubnetConfig
from nncf.experimental.torch.nas.bootstrapNAS.elasticity.elasticity_dim import ElasticityDim

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

MODEL_DIR = Path("../../../models/supernets/cifar10/resnet50")
OUTPUT_DIR = Path("output")
DATA_DIR = Path("data")
BASE_MODEL_NAME = "resnet-50"

fp32_pth_path, model_onnx_path, supernet_onnx_path, subnet_onnx_path = create_folders_demo(BASE_MODEL_NAME + '_external_search')


['/localdisk/maciej/code/dynast_bootstrapnas_integration/hardware_aware_automated_machine_learning/examples/bootstrapNAS/third_party_search', '/localdisk/maciej/miniconda3/envs/bnas_new/lib/python39.zip', '/localdisk/maciej/miniconda3/envs/bnas_new/lib/python3.9', '/localdisk/maciej/miniconda3/envs/bnas_new/lib/python3.9/lib-dynload', '', '/localdisk/maciej/miniconda3/envs/bnas_new/lib/python3.9/site-packages', '..', '/localdisk/maciej/code/dynast_bootstrapnas_integration/nncf/']
Imported PyTorch and NNCF
Using cuda device


<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Loading an existing super-network
</h3>

In [3]:
model = resnet50_cifar10()
config = SampleConfig.from_json(os.path.join(MODEL_DIR, 'config.json'))
config.device = device
print(MODEL_DIR)
supernet = TrainedSuperNet.from_checkpoint(model, config, os.path.join(MODEL_DIR, 'elasticity.pth'), os.path.join(MODEL_DIR, 'supernet_weights.pth'))

../../../models/supernets/cifar10/resnet50


<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Explore the Search Space
</h3>



In [4]:
supernet.get_search_space()

{'width': {0: [256, 200, 152, 128],
  1: [512, 408, 304, 256],
  2: [1024, 816, 608, 512],
  3: [2048, 1632, 1224, 1024],
  4: [64, 48, 32],
  5: [64, 48, 32],
  6: [64, 48, 32],
  7: [64, 48, 32],
  8: [64, 48, 32],
  9: [64, 48, 32],
  10: [64, 48, 32],
  11: [128, 96, 72, 64],
  12: [128, 96, 72, 64],
  13: [128, 96, 72, 64],
  14: [128, 96, 72, 64],
  15: [128, 96, 72, 64],
  16: [128, 96, 72, 64],
  17: [128, 96, 72, 64],
  18: [128, 96, 72, 64],
  19: [256, 200, 152, 128],
  20: [256, 200, 152, 128],
  21: [256, 200, 152, 128],
  22: [256, 200, 152, 128],
  23: [256, 200, 152, 128],
  24: [256, 200, 152, 128],
  25: [256, 200, 152, 128],
  26: [256, 200, 152, 128],
  27: [256, 200, 152, 128],
  28: [256, 200, 152, 128],
  29: [256, 200, 152, 128],
  30: [256, 200, 152, 128],
  31: [512, 408, 304, 256],
  32: [512, 408, 304, 256],
  33: [512, 408, 304, 256],
  34: [512, 408, 304, 256],
  35: [512, 408, 304, 256],
  36: [512, 408, 304, 256]},
 'depth': [[0],
  [2],
  [4],
  [5],
  

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Data Loaders for validation and batchnorm adaptation
</h3>

In [5]:
DATASET_DIR = DATA_DIR / "cifar10"

batch_size_val = 1000
batch_size = 64
train_loader, val_loader = create_cifar10_dataloader(DATASET_DIR, batch_size, batch_size_val, device)

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Sample sub-networks to implement your own algorithm
</h3>


In [6]:
# Implement your function to validate 
def acc_top1(model):
    from bootstrapnas_utils import validate
    model.eval()
    model.to(device)
    model_top1_acc, _, _ = validate(model, val_loader) 
    return model_top1_acc

supernet.activate_maximal_subnet()
supernet.eval_active_subnet(acc_top1)

Test: [ 0/10]	Time 1.673 (1.673)	Loss 0.546 (0.546)	Acc@1 86.90 (86.90)	Acc@5 98.10 (98.10)
 * Acc@1 87.950 Acc@5 98.420


tensor(87.9500, device='cuda:0')

In [7]:
subnet_config = SubnetConfig()
subnet_config[ElasticityDim.DEPTH] = [1]

supernet.eval_subnet(subnet_config, acc_top1)

Test: [ 0/10]	Time 0.591 (0.591)	Loss 0.546 (0.546)	Acc@1 86.90 (86.90)	Acc@5 98.10 (98.10)
 * Acc@1 87.950 Acc@5 98.420


tensor(87.9500, device='cuda:0')

In [8]:
supernet.activate_config(supernet.get_search_space())
supernet.get_macs_for_active_config()

325.799936

In [9]:
supernet.activate_config(supernet.get_search_space())
supernet.get_macs_for_active_config()

325.799936

In [10]:
subnet_config[ElasticityDim.DEPTH] = [0]
subnet_config[ElasticityDim.WIDTH] = {2: 512}

supernet.activate_config(subnet_config)
supernet.get_macs_for_active_config()

272.32256

In [11]:
supernet.get_active_config()

OrderedDict([(<ElasticityDim.WIDTH: 'width'>,
              {0: 256,
               1: 512,
               2: 512,
               3: 2048,
               4: 64,
               5: 64,
               6: 64,
               7: 64,
               8: 64,
               9: 64,
               10: 64,
               11: 128,
               12: 128,
               13: 128,
               14: 128,
               15: 128,
               16: 128,
               17: 128,
               18: 128,
               19: 256,
               20: 256,
               21: 256,
               22: 256,
               23: 256,
               24: 256,
               25: 256,
               26: 256,
               27: 256,
               28: 256,
               29: 256,
               30: 256,
               31: 512,
               32: 512,
               33: 512,
               34: 512,
               35: 512,
               36: 512}),
             (<ElasticityDim.DEPTH: 'depth'>, [0])])

In [12]:
supernet.eval_subnet(subnet_config, acc_top1)

Test: [ 0/10]	Time 0.571 (0.571)	Loss 0.473 (0.473)	Acc@1 88.90 (88.90)	Acc@5 98.40 (98.40)
 * Acc@1 89.510 Acc@5 98.680


tensor(89.5100, device='cuda:0')

<h3 style="text-align: center; background-color: rgb(36, 24, 142); color: white; border: 4px solid rgb(36, 24, 142);
border-radius: 25px;">Search using DyNAS-t
</h3>
TODO



In [13]:
%load_ext autoreload
%autoreload 2
## This cell will be moved to scripts/utils

from typing import Dict
from nncf.common.initialization.batchnorm_adaptation import BatchnormAdaptationAlgorithm
from dynast.utils.nn import validate_classification
from dynast.dynast_manager import DyNAS
from pprint import pprint

def create_dynast_config(nncf_config: Dict, bootstrapnas_supernetwork: TrainedSuperNet):
    search_tactic = 'linas'
    if 'random' in search_tactic:
        dynast_config = {
            'search_tactic': 'random',
            'results_path': 'bootstrapnas_resnet50_cifar10_random_test_1.csv',
            'num_evals': 500,
            'population': 500,
        }
    elif 'linas' in search_tactic:
        dynast_config = {
            'search_tactic': 'linas',
            'results_path': 'bootstrapnas_resnet50_cifar10_linas_test_nb_1.csv',
            'num_evals': 250,
            'population': 50,
        }

    dynast_config.update(
        {
            'seed': nncf_config.seed if nncf_config.seed else 42,
            'supernet': 'bootstrapnas_image_classification',
            'test_fraction': 1.0,
            'optimization_metrics': ['accuracy_top1', 'macs'],
            'measurements': ['accuracy_top1', 'macs'],
            'batch_size': nncf_config.batch_size,
            'dataset_path': DATASET_DIR,
            'bootstrapnas_supernetwork': bootstrapnas_supernetwork,  # This is the only new param that has to be passed
            'device': nncf_config.device,
            'verbose': False,
        }
    )
    return dynast_config

dynast_config = create_dynast_config(config, supernet)

bn_adapt_args = BNAdaptationInitArgs(data_loader=wrap_dataloader_for_init(train_loader), device=config.device)

pprint(config)

bn_adapt_algo_kwargs = {
    'data_loader': train_loader,
    'num_bn_adaptation_samples': 6000,
    'device': 'cuda',
}
bn_adaptation = BatchnormAdaptationAlgorithm(**bn_adapt_algo_kwargs)


def accuracy_top1_fn(_model):
    bn_adaptation.run(_model)

    losses, top1, top5 = validate_classification(
        model=_model,
        data_loader=val_loader,
        device=config.device,
    )
    return top1

dynast_config['metric_eval_fns'] = {
    'accuracy_top1': accuracy_top1_fn,
}

dynas = DyNAS(**dynast_config)
dynas.search()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[06-27 12:34:18 MainProcess #58826] INFO  dynast_manager.py:44 - ========================================
[06-27 12:34:18 MainProcess #58826] INFO  dynast_manager.py:45 - Starting Dynamic NAS Toolkit (DyNAS-T)
[06-27 12:34:18 MainProcess #58826] INFO  dynast_manager.py:46 - ========================================
[06-27 12:34:18 MainProcess #58826] INFO  dynast_manager.py:72 - ----------------------------------------
[06-27 12:34:18 MainProcess #58826] INFO  dynast_manager.py:73 - DyNAS Parameter Inputs:
[06-27 12:34:18 MainProcess #58826] INFO  dynast_manager.py:75 - search_tactic: linas
[06-27 12:34:18 MainProcess #58826] INFO  dynast_manager.py:75 - results_path: bootstrapnas_resnet50_cifar10_linas_test_nb_1.csv
[06-27 12:34:18 MainProcess #58826] INFO  dynast_manager.py:75 - num_evals: 250
[06-27 12:34:18 MainProcess #58826] INFO  dynast_manager.py:75 - population: 50
[06-27 12:34:18 MainProcess #58826] INFO  dynast_manager.py:75 - seed: 42
[06-27 12:34:18 MainProcess #58826] INFO

{'batch_size': 64,
 'batch_size_val': 2000,
 'bootstrapNAS': {'search': {'algorithm': 'DyNAS-T',
                             'batchnorm_adaptation': {'num_bn_adaptation_samples': 6000},
                             'num_evals': 1,
                             'population': 1,
                             'ref_acc': 93.65,
                             'search_tactic': 'random'},
                  'training': {'algorithm': 'progressive_shrinking',
                               'batchnorm_adaptation': {'num_bn_adaptation_samples': 1500},
                               'elasticity': {'available_elasticity_dims': ['width',
                                                                            'depth'],
                                              'depth': {'mode': 'manual',
                                                        'skipped_blocks': [['ResNet/Sequential[layer1]/Bottleneck[1]/ReLU[relu]/relu__2',
                                                                          

[06-27 12:34:19 MainProcess #58826] INFO  __init__.py:81 - > Finished validation_dataloader in 0.7167 s
[06-27 12:34:20 MainProcess #58826] INFO  bootstrapnas_interface.py:90 - Using custom accuracy_top1 metric evaluation function.
[06-27 12:34:26 MainProcess #58826] INFO  __init__.py:76 - > Calling validate_classification
[06-27 12:34:27 MainProcess #58826] INFO  __init__.py:81 - > Finished validate_classification in 1.5901 s
[06-27 12:34:27 MainProcess #58826] INFO  bootstrapnas_interface.py:108 - Validation accuracy: 92.98%
[06-27 12:34:28 MainProcess #58826] INFO  __init__.py:76 - > Calling get_macs
[06-27 12:34:29 MainProcess #58826] INFO  __init__.py:81 - > Finished get_macs in 0.8732 s
[06-27 12:34:29 MainProcess #58826] INFO  search_tactic.py:367 - Evaluating subnetwork 2/50 in loop 1 of 5
[06-27 12:34:29 MainProcess #58826] INFO  __init__.py:76 - > Calling validation_dataloader


Files already downloaded and verified


...

Files already downloaded and verified


[06-27 13:19:08 MainProcess #58826] INFO  __init__.py:81 - > Finished validation_dataloader in 0.7232 s
[06-27 13:19:09 MainProcess #58826] INFO  bootstrapnas_interface.py:90 - Using custom accuracy_top1 metric evaluation function.
[06-27 13:19:15 MainProcess #58826] INFO  __init__.py:76 - > Calling validate_classification
[06-27 13:19:17 MainProcess #58826] INFO  __init__.py:81 - > Finished validate_classification in 1.4550 s
[06-27 13:19:17 MainProcess #58826] INFO  bootstrapnas_interface.py:108 - Validation accuracy: 93.39%
[06-27 13:19:17 MainProcess #58826] INFO  __init__.py:76 - > Calling get_macs
[06-27 13:19:18 MainProcess #58826] INFO  __init__.py:81 - > Finished get_macs in 0.9746 s
[06-27 13:19:18 MainProcess #58826] INFO  search_tactic.py:345 - Training macs predictor.
[06-27 13:19:18 MainProcess #58826] INFO  predictor_manager.py:38 - Building dynamic predictor for macs
[06-27 13:19:18 MainProcess #58826] INFO  encoding.py:246 - Training set length=250
[06-27 13:19:24 Main

....................................................................................................

[06-27 13:19:29 MainProcess #58826] INFO  evolutionary.py:350 - Success! Search Took 4.227 seconds.
[06-27 13:19:29 MainProcess #58826] INFO  search_tactic.py:526 - Validated model architectures in file: bootstrapnas_resnet50_cifar10_linas_test_nb_1.csv


[OrderedDict([(<ElasticityDim.WIDTH: 'width'>,
               {0: 128,
                1: 256,
                2: 512,
                3: 1024,
                4: 32,
                5: 32,
                6: 32,
                7: 32,
                8: 32,
                9: 32,
                10: 48,
                11: 64,
                12: 64,
                13: 64,
                14: 64,
                15: 72,
                16: 64,
                17: 64,
                18: 64,
                19: 152,
                20: 128,
                21: 200,
                22: 152,
                23: 128,
                24: 128,
                25: 200,
                26: 128,
                27: 152,
                28: 128,
                29: 128,
                30: 152,
                31: 304,
                32: 256,
                33: 256,
                34: 304,
                35: 256,
                36: 256}),
              (<ElasticityDim.DEPTH: 'depth'>, [0,